Evaluation of the Fashion-MNIST Database Using Ensemble-based XGBoost and Neural Network-based TensorFlow Models

Author: Prabode Weebadde

Note:

To install GPU support on Macbook Pro M2 Max chip follow the excellent tutorial by J. Hanson.
https://www.youtube.com/watch?v=5DgWvU0p2bk

Credits:

https://github.com/moritzhambach/CPU-vs-GPU-benchmark-on-MNIST/blob/master/gputest.ipynb

In [1]:
import tensorflow as tf
import numpy as np                                
import matplotlib.pyplot as plt
import keras as k
# from tensorflow.examples.tutorials.mnist import input_data
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from keras import backend as K


In [4]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [23]:

# from tensorflow.examples.tutorials.mnist import input_data
x_train, y_train = load_mnist('data/fashion', kind='train')
x_test, y_test = load_mnist('data/fashion', kind='t10k')

# data preprocessing

img_rows, img_cols = 28,28
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

input_shape = (img_rows, img_cols, 1)
x_test=x_test.astype('float32')
x_train=x_train.astype('float32')
mean=np.mean(x_train)
std=np.std(x_train)
x_test = (x_test-mean)/std
x_train = (x_train-mean)/std

# labels
num_classes=10
y_train = k.utils.to_categorical(y_train, num_classes)
y_test = k.utils.to_categorical(y_test, num_classes)

In [8]:
# for i in range(8):

n=16

# build model
num_filter=n
num_dense=512
drop_dense=0.7
ac='relu'
learningrate=0.001

model = Sequential()

model.add(Conv2D(num_filter, (3, 3), activation=ac, input_shape=(28, 28, 1), padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(num_filter, (3, 3), activation=ac, padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 14x14x32

model.add(Conv2D(2*num_filter, (3, 3), activation=ac, padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(2*num_filter, (3, 3), activation=ac, padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 7x7x64 = 3136 neurons

model.add(Flatten())                        
model.add(Dense(num_dense, activation=ac))
model.add(BatchNormalization())
model.add(Dropout(drop_dense))
model.add(Dense(10, activation='softmax'))

adm=Adam(learning_rate=learningrate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adm)


Metal device set to: Apple M2 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



In [9]:

batch=2048
epochz=200
print("number of filters " + str(n))
print("batch size " + str(batch))
model.fit(x_train, y_train, batch_size=batch, epochs=epochz, validation_data=(x_test, y_test))

number of filters 16
batch size 2048
Epoch 1/200


2023-04-23 16:47:22.322953: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


30/30 [==============================] - 12s 132ms/step - loss: 0.8523 - accuracy: 0.7549 - val_loss: 1.9021 - val_accuracy: 0.3222
Epoch 2/200
30/30 [==============================] - 2s 56ms/step - loss: 0.4131 - accuracy: 0.8569 - val_loss: 2.0232 - val_accuracy: 0.2054
Epoch 3/200
30/30 [==============================] - 2s 57ms/step - loss: 0.3408 - accuracy: 0.8775 - val_loss: 2.1484 - val_accuracy: 0.2001
Epoch 4/200
30/30 [==============================] - 2s 60ms/step - loss: 0.3028 - accuracy: 0.8910 - val_loss: 1.9566 - val_accuracy: 0.2481
Epoch 5/200
30/30 [==============================] - 2s 62ms/step - loss: 0.2754 - accuracy: 0.9003 - val_loss: 1.8213 - val_accuracy: 0.3440
Epoch 6/200
30/30 [==============================] - 2s 59ms/step - loss: 0.2558 - accuracy: 0.9074 - val_loss: 1.5002 - val_accuracy: 0.4966
Epoch 7/200
30/30 [==============================] - 2s 58ms/step - loss: 0.2376 - accuracy: 0.9143 - val_loss: 1.3193 - val_accuracy: 0.6129
Epoch 8/200
30/3

In [10]:
model.save('saved_model/tf_model')

INFO:tensorflow:Assets written to: saved_model/tf_model/assets


INFO:tensorflow:Assets written to: saved_model/tf_model/assets


In [11]:
new_model = tf.keras.models.load_model('saved_model/tf_model')


In [12]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 16)       64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 16)        2320      
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 16)       64        
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                        

In [13]:
loss, acc = new_model.evaluate(x_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(x_test).shape)

313/313 - 4s - loss: 0.6167 - accuracy: 0.9197 - 4s/epoch - 13ms/step
Restored model, accuracy: 91.97%
313/313 [==============================] - 2s 4ms/step
(10000, 10)


In [25]:

xx_test, yy_test = load_mnist('data/fashion', kind='t10k')
test_predictions = np.argmax(new_model.predict(x_test), axis=-1)
print('generating classification report.....')
print(classification_report(yy_test,test_predictions))

313/313 [==============================] - 1s 2ms/step
generating classification report.....
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      1000
           1       0.99      0.98      0.99      1000
           2       0.86      0.89      0.87      1000
           3       0.91      0.92      0.91      1000
           4       0.86      0.87      0.86      1000
           5       0.99      0.98      0.98      1000
           6       0.79      0.76      0.77      1000
           7       0.96      0.98      0.97      1000
           8       0.99      0.98      0.98      1000
           9       0.98      0.97      0.97      1000

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000

